In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.result import marginal_counts
from qiskit.circuit.library import RXGate, CXGate, CZGate
from TN_circuit import*

ModuleNotFoundError: No module named 'tenpy.networks'

In [ ]:
n_qubits = 10
mps_order = list(range(n_qubits))
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
Id = np.eye(2)

In [ ]:
qc = QuantumCircuit(n_qubits, n_qubits)
qc.rx(np.pi/4, 3)
qc.cx(3,5)
qc.x(7)
qc.x(9)
qc.cz(7,9)
for i in range(n_qubits):
    qc.measure(i,i)
qc.draw(output='mpl')

In [ ]:
shots = 10000
backend = AerSimulator()
job = backend.run(qc, shots=shots)
counts = job.result().get_counts()

In [ ]:
z_exp_vals = []
for i in range(n_qubits):
    m_counts = marginal_counts(counts, [i])
    z_exp_vals.append((m_counts.get('0', 0)-m_counts.get('1', 0))/shots)
    

In [ ]:
rx_pi4 = RXGate(np.pi/4).to_matrix()
tn_circ = TNCircuit(mps_order, chi_max=200)
tn_circ.initialize_state(0)
tn_circ.apply_single_q_gate(4, rx_pi4)
tn_circ.apply_controlled_gates(3,5,X)
tn_circ.apply_single_q_gate(7,'x')
tn_circ.apply_single_q_gate(9, 'x')
tn_circ.apply_controlled_gates(7,9,Z)

z_exp_vals_tn = []
for i in range(n_qubits):
    z_exp_vals_tn.append(tn_circ.meas_product_op_expectation_val(i, ['Sigmaz']))

In [ ]:
print(z_exp_vals)
print(z_exp_vals_tn)